Install [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html)

In [ ]:
import gzip
import io
import json
import os
import pprint
import re
import subprocess
import time
from zipfile import ZipFile
from datetime import timedelta, datetime
from pathlib import Path
from dataclasses import dataclass, asdict
import multiprocessing

import numpy as np
import pandas as pd
import requests

In [ ]:
@dataclass
class DataSetConfig:
    content_path: Path
    embedding_path: Path
    num_of_docs: int
    index_size: int
    bulk_size: int
    index_name: str
    distance: str
    dimension: int
    exact: bool
    hnsw_m: int
    hnsw_ef_construction: int
    hnsw_ef: int
    update_docs_per_sec: int
    quantization: str

    vespa_name: str = "benchmark_vespa"
    vespa_host: str = "localhost"
    vespa_port: int = 8090
    vespa_management_port: int = 19081
    vespa_version: str = "8.472.109"


def get_dataset_config(target_name):
    setting = {
        "100k-768-m32-efc200-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 100000,
            "bulk_size": 1000,
            "index_name": "contents",
            "distance": "dotproduct", # "angular"
            "dimension": 768,
            "exact": False,
            "hnsw_m": 32,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
            "quantization": "float", # "bfloat16",
        },
        "1m-768-m48-efc200-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 1000000,
            "bulk_size": 1000,
            "index_name": "contents",
            "distance": "dotproduct", # "angular"
            "dimension": 768,
            "exact": False,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
            "quantization": "float", # "bfloat16",
        },
        "5m-768-m48-efc200-ef100-ip": {
            "content_path": Path("dataset/passages-c400-jawiki-20230403"),
            "embedding_path": Path("dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage"),
            "num_of_docs": 5555583,
            "index_size": 5000000,
            "bulk_size": 1000,
            "index_name": "contents",
            "distance": "dotproduct", # "angular"
            "dimension": 768,
            "exact": False,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
            "update_docs_per_sec": 0,
            "quantization": "float", # "bfloat16",
        },
    }
    return DataSetConfig(**setting.get(target_name))


In [ ]:
def run_vespa(config):
    volume_dir = os.getenv("VOLUME_DIR", "./data")
    print(f"Starting {config.vespa_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "run", "-d",
        "--name", config.vespa_name,
        "-p", f"{config.vespa_port}:8080",
        "-p", f"{config.vespa_management_port}:19071",
        # "-v", f"{volume_dir}:/opt/vespa/var",
        f"vespaengine/vespa:{config.vespa_version}"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def stop_vespa(config):
    print(f"Stopping {config.vespa_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "stop", config.vespa_name
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def prune_docker(config):
    print(f"Cleaning up... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "system", "prune", "-f"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def print_docker_system_df(config):
    docker_cmd = [
        # "sudo",
        "docker", "system", "df"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print(result.stderr)


In [ ]:
def print_docker_container_stats(config):
    docker_cmd = [
        # "sudo",
        "docker", "container", "stats", "--no-stream"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    containers = {}
    if result.returncode == 0:
        print(result.stdout)
        for line in result.stdout.split("\n"):
            if line.startswith("CONTAINER") or len(line) == 0:
                continue
            values = line.split()
            containers[values[1]] = {
                "container_id": values[0],
                "cpu": values[2],
                "mem": values[6],
                "mem_usage": values[3],
                "mem_limit": values[5],
                "net_in": values[7],
                "net_out": values[9],
                "block_in": values[10],
                "block_out": values[12],
                "pids": values[13],
            }
    else:
        print(result.stderr)
    return containers


In [ ]:
def create_index(config):
    service_xml_str = f"""<?xml version='1.0' encoding='UTF-8'?>
<services version="1.0" xmlns:deploy="vespa" xmlns:preprocess="properties">
  <container id='default' version='1.0'>
    <search></search>
    <document-api></document-api>
    <nodes>
      <node hostalias='node1'></node>
    </nodes>
  </container>
  <content id='wikipedia' version='1.0'>
    <redundancy>2</redundancy>
    <documents>
      <document type="{config.index_name}" mode="index"/>
    </documents>
    <nodes>
      <node hostalias="node1" distribution-key="0" />
    </nodes>
    <tuning>
      <resource-limits>
        <disk>0.95</disk>
        <memory>0.95</memory>
      </resource-limits>
    </tuning>
  </content>
</services>
"""

    sd_str = """
schema {index_name} {
    document {index_name} {
        field page_id type int {
            indexing: attribute | summary
        }
        field rev_id type int {
            indexing: attribute | summary
        }
        field title type string {
            indexing: index | summary
            index: enable-bm25
        }
        field section type string {
            indexing: attribute | summary
            attribute: fast-search
        }
        field text type string {
            indexing: index | summary
            index: enable-bm25
        }
        field embedding type tensor<{float_type}>(x[{dimension}]) {
            indexing: attribute | index
            attribute {
                distance-metric: {distance}
            }
            index {
                hnsw {
                    max-links-per-node: {hnsw_m}
                    neighbors-to-explore-at-insert: {hnsw_ef_construction}
                }
            }
        }
    }

    fieldset default {
        fields: title,text
    }

    rank-profile default {
        first-phase {
            expression: nativeRank(title, text)
        }
    }

    rank-profile closeness {
        num-threads-per-search: 1
        match-features: distance(field, embedding)

        inputs {
            query(q)  tensor<{float_type}>(x[{dimension}])
            query(qa) tensor<{float_type}>(x[{dimension}])
        }

        first-phase {
            expression: closeness(field, embedding)
        }
    }
}
""".replace("{index_name}", config.index_name)\
    .replace("{distance}", str(config.distance))\
    .replace("{dimension}", str(config.dimension))\
    .replace("{hnsw_m}", str(config.hnsw_m))\
    .replace("{hnsw_ef_construction}", str(config.hnsw_ef_construction))\
    .replace("{float_type}", config.quantization)

    query_profile_str = """
<query-profile id="default">
    <field name="presentation.timing">true</field>
</query-profile>
"""

    zip_buffer = io.BytesIO()
    with ZipFile(zip_buffer, 'w') as zip_file:
        zip_file.writestr("services.xml", service_xml_str)
        zip_file.writestr(f"schemas/{config.index_name}.sd", sd_str)
        zip_file.writestr("search/query-profiles/default.xml", query_profile_str)
    zip_buffer.seek(0)
    
    print(F"Creating {config.index_name}... ", end="")
    response = requests.post(f"http://{config.vespa_host}:{config.vespa_management_port}/application/v2/tenant/default/prepareandactivate",
                             headers={"Content-Type": "application/zip"},
                             data=zip_buffer)
    if response.status_code == 200:
        print("[OK]")
    else:
        print(f"[FAIL]\n{response.text}")


In [ ]:
def delete_index(config):
    print(F"Deleting {config.index_name}... ", end="")
    response = requests.delete(f"http://{config.vespa_management_port}:{config.vespa_port}/application/v2/tenant/default")
    if response.status_code == 200:
        print("[OK]")
    else:
        print(f"[FAIL]\n{response.text}")


In [ ]:
def print_indices(config):
    response = requests.get(f"http://{config.vespa_host}:{config.vespa_port}/search/",
                            headers={"Content-Type": "application/json"},
                            params={
                                "yql": "select * from sources * where sddocname contains 'contents';"
                            })
    obj = json.loads(response.text)
    count = obj.get('root').get('fields').get('totalCount')
    print(f"count: {count}")
    return {
        "num_of_docs": count,
    }


In [ ]:
def wait_for_index(config):
    while True:
        try:
            response = requests.get(f"http://{config.vespa_host}:{config.vespa_port}/search/",
                                    headers={"Content-Type": "application/json"},
                                    params={
                                        "yql": "select * from sources * where sddocname contains 'contents';"
                                    })
            if response.status_code == 200:
                break
        except:
            pass
        print(".", end="")
        time.sleep(1)
    print(".")


In [ ]:
def wait_for_vespa(config, retry_count=60):
    print(f"Waiting for {config.vespa_name}", end="")
    for i in range(retry_count):
        try:
            response = requests.get(f"http://{config.vespa_host}:{config.vespa_management_port}/state/v1/health")
            if response.status_code == 200:
                obj = json.loads(response.text)
                if obj.get("status").get("code") == "up":
                    print("[OK]")        
                    return
        except:
            pass
        print(".", end="")
        time.sleep(1)
    print("[FAIL]")


In [ ]:
def get_embedding(config, embedding_index, embedding_data, id):
    emb_index = int(id / 100000) * 100000
    if embedding_data is None or embedding_index != emb_index:
        with np.load(config.embedding_path / f"{emb_index}.npz") as data:
            embedding_data = data["embs"]
    embedding = embedding_data[id - emb_index]
    if config.distance == "dotproduct":
        embedding = embedding / np.linalg.norm(embedding)
        embedding = embedding.astype(np.float32)
    return emb_index, embedding_data, embedding


def check_error_counts(config, s):
    regex = r'"http\.response\.error\.count": (\d+)'
    matches = re.findall(regex, s)
    for count in [int(match) for match in matches]:
        if count > 0:
            return False
    return True


def get_section_values(config, df, min_count=10000):
    tmp_df = df[["id","section"]].groupby("section").count().reset_index()
    tmp_df = tmp_df[tmp_df["id"] >= min_count]
    return tmp_df["section"].values.tolist()


def insert_data(config, max_size, bulk_size, controller=None, query_data=None):
    start_time = time.time()

    docs = []
    def send_data(pos):
        print(F"Sending {int(len(docs))} docs ({pos}/{max_size})... ", end="")
        with open("vespa_docs.jsonl", "wt") as f:
            for doc in docs:
                f.write(doc)
                f.write("\n")
        # use vespa command
        vespa_cmd = [
            "vespa", "feed", "vespa_docs.jsonl",
            "--target", f"http://{config.vespa_host}:{config.vespa_port}"
        ]
        now = time.time()
        result = subprocess.run(vespa_cmd, capture_output=True, text=True)
        if result.returncode == 0 and check_error_counts(config, result.stdout):
            t = time.time() - now
            print(f"[OK] {t}")
            return t
        else:
            print(f"[FAIL] 0 {result.returncode} STDOUT:{result.stdout} STDERR:{str(result.stderr)[0:10000]}")
            return 0

    total_time = 0
    count = 0
    embedding_index = -1
    embedding_data = None
    for content_file in sorted(config.content_path.glob("*.parquet")):
        if count >= max_size:
            break
        df = pd.read_parquet(content_file)
        if query_data is not None:
            query_data["section_values"].extend(get_section_values(config, df))
        for i,row in df.iterrows():
            if count >= max_size:
                break
            embedding_index, embedding_data, embedding = get_embedding(config, embedding_index, embedding_data, row.id)
            count += 1
            docs.append(json.dumps({
                "put": f"id:{config.index_name}:{config.index_name}::{count}",
                "fields": {
                    "page_id": row.pageid,
                    "rev_id": row.revid,
                    # "title": row.title,
                    "section": row.section,
                    # "text": row.text,
                    "embedding": embedding.tolist(),
                }
            }))
            if len(docs) >= bulk_size:
                total_time += send_data(count)
                docs = []
                if controller is not None and not controller.run():
                    return

    if len(docs) > 0:
        total_time += send_data(count)

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f} ({timedelta(seconds=total_time)})")
    return {
        "execution_time": execution_time,
        "process_time": total_time,
    }


In [ ]:
class IndexingController:

    def __init__(self, stop_event):
        self._previous_time = time.time()
        self._stop_event = stop_event

    def run(self):
        if self._stop_event.is_set():
            return False

        wait_time = 1 - ( time.time() - self._previous_time)
        if wait_time > 0:
            # print(f"Waiting for {wait_time} sec")
            time.sleep(wait_time)
        self._previous_time =  time.time()
        return True


def update_data(target_config, stop_event):
    dataset_config = get_dataset_config(target_config)
    if dataset_config.update_docs_per_sec > 0:
        print(f"Starting update for {target_config}")
        insert_data(dataset_config, 
                    max_size=dataset_config.index_size,
                    bulk_size=dataset_config.update_docs_per_sec,
                    controller=IndexingController(stop_event))
        print(f"Stopping update for {target_config}")
    else:
        print(f"No background updates")


def start_update(target_config):
    stop_event = multiprocessing.Event()

    p = multiprocessing.Process(target=update_data, args=(target_config, stop_event))
    p.start()

    def stop_update():
        stop_event.set()
        p.join()

    return stop_update


In [ ]:
def search(config, query):
    response = requests.post(f"http://{config.vespa_host}:{config.vespa_port}/search/",
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(query))
    # print(response.text)

    if response.status_code == 200:
        obj = json.loads(response.text)
        took = obj.get("timing").get("searchtime") * 1000
        hits = obj.get("root").get("children")
        if hits is None:
            hits = []
        product_ids = [int(x.get("id").split(":")[-1]) for x in hits]
        scores = [x.get("relevance") for x in hits]
        return took, len(hits), int(obj.get("root").get("coverage").get("documents")), product_ids, scores
    print(f"[FAIL][{response.status_code}] {response.text}")
    return -1, -1, -1, [], []


In [ ]:
def search_with_knn_queries(config, output_path, pre_filter=None, max_size=10000, page_size=100, offset=0, max_error_count=100, exact=False):
    print("Sending knn queries...")
    start_time = time.time()
    pos = offset
    doc_id = 0
    count = 0
    running = True
    error_count = 0
    with gzip.open(output_path, "wt", encoding="utf-8") as f:
        while running:
            with np.load(config.embedding_path / f"{pos}.npz") as data:
                embedding_data = data["embs"]
            for embedding in embedding_data:
                doc_id += 1
                if count >= max_size:
                    running = False
                    break
                if config.distance == "dotproduct":
                    embedding = embedding.astype(np.float32)
                    embedding = embedding / np.linalg.norm(embedding)
                target_hits = config.hnsw_ef if config.hnsw_ef > page_size else page_size
                approximate = "true" if not exact else "false"
                yql =  f"""
                select documentid from {config.index_name}
                where {{approximate:{approximate},targetHits:{target_hits}}}nearestNeighbor(embedding,q)
                """.strip()
                if pre_filter is not None:
                    yql = f"{yql} and {next(pre_filter)}"
                # print(yql)
                query = {
                    "hits": page_size,
                    "yql": yql,
                    "ranking": "closeness",
                    "input.query(q)": embedding.tolist(),
                }
                took, hits, total_hits, ids, scores = search(config, query=query)
                # print(f"{took}, {total_hits}, {ids}, {scores}")
                if took == -1:
                    error_count += 1
                    if error_count >= max_error_count:
                        running = False
                        break
                    continue
                result = {
                    "id": doc_id,
                    "took": took,
                    "hits": hits,
                    "total_hits": total_hits,
                    "ids": ids,
                    "scores": scores,
                }
                f.write(json.dumps(result, ensure_ascii=False))
                f.write("\n")
                count += 1
                if count % 10000 == 0:
                    print(f"Sent {count}/{max_size} queries.")

            pos += 100000
            if pos > config.num_of_docs:
                pos = 0

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f}")


In [ ]:
def get_output_filename(version, name, exact):
    filename = f"output/vespa{version.replace('.', '_')}_{name}"
    if exact:
        filename += "_exact"
    filename += ".jsonl.gz"
    return filename


In [ ]:
def print_took_and_total_hits(k, filename, truth_filename):
    query_ids = []
    tooks = []
    hits = []
    df = pd.read_json(filename, lines=True)
    result = {
        "num_of_queries": len(df),
        "took": {
            "mean": df.took.mean(),
            "std": df.took.std(),
            "min": df.took.min(),
            "25%": df.took.quantile(0.25),
            "50%": df.took.quantile(0.5),
            "75%": df.took.quantile(0.75),
            "90%": df.took.quantile(0.9),
            "99%": df.took.quantile(0.99),
            "max": df.took.max(),
        },
        "hits": {
            "mean": df.hits.mean(),
            "std": df.hits.std(),
            "min": df.hits.min(),
            "25%": df.hits.quantile(0.25),
            "50%": df.hits.quantile(0.5),
            "75%": df.hits.quantile(0.75),
            "max": df.hits.max(),
        },
    }
    if os.path.exists(truth_filename):
        df = pd.merge(df,
                      pd.read_json(truth_filename, lines=True)[["id", "ids"]].rename(columns={"ids":"truth_ids"}),
                      on="id", how="inner")
        def get_precision(row):
            size = len(row["truth_ids"])
            if size > k:
                size = k
            return len(set(row["ids"]).intersection(set(row["truth_ids"]))) / size
        df["precision"] = df.apply(get_precision, axis=1)
        result["precision"] = {
            "mean": df.precision.mean(),
            "std": df.precision.std(),
            "min": df.precision.min(),
            "25%": df.precision.quantile(0.25),
            "50%": df.precision.quantile(0.5),
            "75%": df.precision.quantile(0.75),
            "90%": df.precision.quantile(0.9),
            "99%": df.precision.quantile(0.99),
            "max": df.precision.max(),
        }
    print(df.describe().to_markdown())
    return result


In [ ]:
def save_results(target_config, config, results):
    with open("results.json", "wt", encoding="utf-8") as f:
        json.dump({
            "target": target_config,
            "version": config.vespa_version,
            "settings": asdict(config),
            "results": results,
            "timestamp": datetime.now().isoformat(),
        }, f, ensure_ascii=False, default=lambda x: int(x) if isinstance(x, np.int64) else None)


In [ ]:
query_data = {"section_values": []}
results = {}
target_config = os.getenv("TARGET_CONFIG", "100k-768-m32-efc200-ef100-ip")
dataset_config = get_dataset_config(target_config)
pprint.pprint(dataset_config)

In [ ]:
prune_docker(dataset_config)
print(f"<<<Vespa {dataset_config.vespa_version}>>>")
run_vespa(dataset_config)
wait_for_vespa(dataset_config)

In [ ]:
print_docker_container_stats(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
create_index(dataset_config)
wait_for_index(dataset_config)

In [ ]:
print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
results["indexing"] = insert_data(dataset_config,
                                  max_size=dataset_config.index_size,
                                  bulk_size=dataset_config.bulk_size,
                                  query_data=query_data)

In [ ]:
results["indexing"]["container"] = print_docker_container_stats(dataset_config)
print_indices(dataset_config)
print_docker_system_df(dataset_config)

In [ ]:
for page_size in [10, 100]:
    print(f"page size: {page_size}")
    filename = get_output_filename(dataset_config.vespa_version, f"knn_{page_size}", exact=dataset_config.exact)
    stop_update = start_update(target_config)
    search_with_knn_queries(dataset_config, filename, page_size=page_size, max_size=1000) # warmup
    search_with_knn_queries(dataset_config, filename, page_size=page_size, offset=dataset_config.index_size, exact=dataset_config.exact)
    stop_update()
    results[f"top_{page_size}"] = print_took_and_total_hits(page_size, filename,  f"dataset/ground_truth/{re.sub(r'-m.*', '', target_config)}/knn_{page_size}.jsonl.gz")

In [ ]:
def pre_filter_generator():
    section_values = query_data["section_values"]
    if len(section_values) > 0:
        while True:
            for s in section_values:
                yield f"section contains \"{s}\""

results["num_of_filtered_words"] = len(query_data["section_values"])
for page_size in [10, 100]:
    print(f"page size: {page_size}")
    filename = get_output_filename(dataset_config.vespa_version, f"knn_{page_size}_filtered", exact=dataset_config.exact)
    stop_update = start_update(target_config)
    search_with_knn_queries(dataset_config, filename, page_size=page_size, max_size=1000, pre_filter=pre_filter_generator()) # warmup
    search_with_knn_queries(dataset_config, filename, page_size=page_size, offset=dataset_config.index_size, pre_filter=pre_filter_generator(), exact=dataset_config.exact)
    stop_update()
    results[f"top_{page_size}_filtered"] = print_took_and_total_hits(page_size, filename, f"dataset/ground_truth/{re.sub(r'-m.*', '', target_config)}/knn_{page_size}_filtered.jsonl.gz")

In [ ]:
print_docker_system_df(dataset_config)
print_docker_container_stats(dataset_config)
print_indices(dataset_config)

In [ ]:
save_results(target_config, dataset_config, results)

In [ ]:
# delete_index(dataset_config)
stop_vespa(dataset_config)